## Getting Started with Guardrails AI

In [1]:
!guardrails hub install hub://guardrails/regex_match

Installing hub://guardrails/regex_match...
[==  ] Fetching manifestst
[   =] Downloading dependenciespendencies
[    ] Running post-install setup
✅Successfully installed guardrails/regex_match version 0.0.0!


Import validator:
from guardrails.hub import RegexMatch

Get more info:
https://hub.guardrailsai.com/validator/guardrails/regex_match



In [2]:
!guardrails hub install hub://guardrails/valid_length

Installing hub://guardrails/valid_length...
[  ==] Fetching manifestst
[   =] Downloading dependenciespendencies
[    ] Running post-install setup
✅Successfully installed guardrails/valid_length version 0.0.0!


Import validator:
from guardrails.hub import ValidLength

Get more info:
https://hub.guardrailsai.com/validator/guardrails/valid_length



In [14]:
from pprint import pprint
import os

from langchain_openai import AzureChatOpenAI
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from langchain_groq import ChatGroq


from guardrails.hub import RegexMatch, ValidLength
from guardrails import Guard


In [4]:
pprint(dict(os.environ))

{'APPLICATIONINSIGHTS_CONFIGURATION_CONTENT': '{}',
 'APPLICATION_INSIGHTS_NO_DIAGNOSTIC_CHANNEL': '1',
 'AZURE_API_BASE': 'https://zdx-copilot-ml-dev.openai.azure.com/',
 'AZURE_API_KEY': 'f6a8aced21bc46d28bcf8485dd074883',
 'AZURE_API_VERSION': '2024-02-15-preview',
 'AZURE_ENDPOINT': 'https://zdx-copilot-ml-dev.openai.azure.com/',
 'CLICOLOR': '1',
 'CLICOLOR_FORCE': '1',
 'COLOREDLOGS_LEVEL_STYLES': 'spam=white,faint;success=green,bold;debug=magenta;verbose=blue;notice=cyan,bold;warning=yellow;error=red;critical=background=red',
 'COMMAND_MODE': 'unix2003',
 'CONDA_DEFAULT_ENV': 'base',
 'CONDA_EXE': '/opt/anaconda3/bin/conda',
 'CONDA_PREFIX': '/opt/anaconda3',
 'CONDA_PROMPT_MODIFIER': '(base) ',
 'CONDA_PYTHON_EXE': '/opt/anaconda3/bin/python',
 'CONDA_SHLVL': '1',
 'ELECTRON_RUN_AS_NODE': '1',
 'FORCE_COLOR': '1',
 'GIT_PAGER': 'cat',
 'GROQ_API_KEY': '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',
 'GSETTINGS_SCHEMA_DIR': '/opt/anaconda3/share/glib-2.0/schemas',
 '

In [5]:
gpt_4_o_model = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_ENDPOINT"),
    api_key=os.getenv("OPENAI_API_KEY"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_deployment="gpt-4o",
    model="gpt-4o",
    model_version="2024-05-13",
    temperature=0,
    max_retries=5
)

In [15]:
groq_mistral_model = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=os.getenv("GROQ_API_KEY")
    # other params...
)

In [6]:
gpt_4_o_model.invoke("hi")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-2024-05-13-2024-05-13', 'system_fingerprint': 'fp_04751d0b65', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-5827c86a-6591-4de0-8318-353cafd37a7e-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {}, 'output_token_details': {}})

In [18]:
print(groq_mistral_model.invoke("hi").content)

Hello! How can I help you today? If you have any questions about a specific topic or just want to chat, I'm here for you.

Some popular topics include:

* Programming and software development
* Data science and machine learning
* Technology and the latest gadgets
* Productivity and time management
* Learning and education
* Health and wellness
* Personal finance and investing
* Travel and adventure
* And much more!

Just let me know what's on your mind and I'll do my best to assist you. I'm here to help and make your day a little bit easier. 😊


In [7]:
# initialize the Guard

guard = Guard().use(validator=RegexMatch(regex="^[A-Z][a-z]*$"))

In [8]:
guard.parse("Caesar").validation_passed

/opt/anaconda3/envs/shaunak-dev/lib/python3.12/site-packages/guardrails/validator_service/__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


True

In [10]:
guard.parse("Caesar Salad").validation_passed

ValidationError: Validation failed for field with errors: Result must match ^[A-Z][a-z]*$

In [9]:
guard = Guard().use_many(
    RegexMatch(regex="^[A-Z][a-z]*$"),
    ValidLength(min=0, max=10)
)

guard.parse("Caesar").validation_passed

True

In [10]:
guard.parse("Caesarjsjhskldsklsd").validation_passed

ValidationError: Validation failed for field with errors: Value has length greater than 10. Please return a shorter output, that is shorter than 10 characters.

In [8]:
guard = Guard()

In [9]:
result = guard(
    model="azure/gpt-4o",
    messages=[{"role":"user", "content":"How many moons does Jupiter have?"}],
)

/opt/anaconda3/envs/shaunak-dev/lib/python3.12/site-packages/guardrails/validator_service/__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


In [10]:
result

ValidationOutcome(call_id='4571492544', raw_llm_output="As of my last update in October 2023, Jupiter has 95 known moons. This number can change as new moons are discovered and confirmed, so it's always a good idea to check the latest astronomical data for the most current information.", validation_summaries=[], validated_output="As of my last update in October 2023, Jupiter has 95 known moons. This number can change as new moons are discovered and confirmed, so it's always a good idea to check the latest astronomical data for the most current information.", reask=None, validation_passed=True, error=None)

In [11]:
result = guard(
    model="groq/llama3-8b-8192",
    messages=[{"role":"user", "content":"How many moons does Jupiter have?"}],
)

In [12]:
result

ValidationOutcome(call_id='4672942112', raw_llm_output="A great question about the gas giant!\n\nJupiter, the largest planet in our solar system, has a whopping 79 known moons! These moons range in size from tiny, irregularly shaped bodies to the largest moon, Ganymede, which is even larger than the planet Mercury.\n\nHere are some of the notable moons of Jupiter:\n\n1. Io, Europa, Ganymede, and Callisto (known as the Galilean moons, discovered by Galileo Galilei in 1610)\n2. Amalthea, Thebe, and Metis (small, irregular moons)\n3. Himalia, Elara, and Pasiphae (larger, irregular moons)\n4. Sinope, Leda, and Themisto (distant, irregular moons)\n\nThe majority of Jupiter's moons are small, icy bodies that orbit the planet at distances ranging from 0.1 to 20 million kilometers (0.06 to 12.4 million miles). The largest four moons, Io, Europa, Ganymede, and Callisto, are much larger and more massive, and are thought to have formed from a disk of material that surrounded Jupiter as it formed.

### The Guard

The guard object is the main interface for Guardrails. It can be used without configuration for string-based LLM apps, and accepts a pydantic object for structured data usecases. The Guard is then used to run the Guardrails AI engine. It is the object that wraps LLM calls, orchestrates validation, and keeps track of call history.

__call flow__:

After initializing a guard, you can invoke it using a model name and messages for your prompt, similarly to how you would invoke a regular call to an LLM SDK. Guardrails will call the LLM and then validate the output against the guardrails you've configured. The output will be returned as a GuardResponse object, which contains the raw LLM output, the validated output, and whether or not validation was successful.



In [13]:
res = guard(
    model="groq/llama3-8b-8192",
    messages=[{
        "role": "user",
        "content": "How do I make a cake?"
    }]
)
print(res.raw_llm_output)
print(res.validated_output)
print(res.validation_passed)

Making a cake is a fun and rewarding process! Here's a basic recipe and guide to help you get started:

**Ingredients:**

For a classic vanilla cake, you'll need:

* 2 cups (250g) all-purpose flour
* 1 teaspoon baking powder
* 1 teaspoon salt
* 1 cup (200g) granulated sugar
* 1/2 cup (115g) unsalted butter, softened
* 2 large eggs, at room temperature
* 2 teaspoons pure vanilla extract
* 1 cup (240ml) whole milk, at room temperature

**Equipment:**

* 2 (8-inch/20cm) round cake pans
* Non-stick cooking spray or parchment paper
* Electric mixer (stand or handheld)
* Whisk
* Rubber spatula
* Measuring cups and spoons

**Instructions:**

1. **Preheat your oven:** Preheat your oven to 350°F (180°C).
2. **Prepare the pans:** Grease the cake pans with non-stick cooking spray or line them with parchment paper.
3. **Mix the dry ingredients:** In a medium bowl, whisk together the flour, baking powder, and salt. Set aside.
4. **Mix the wet ingredients:** In a large mixing bowl, use an electric m

__parse__:

If you would rather call the LLM yourself, or at least make the first call yourself, you can use Guard.parse to apply your RAIL specification to the LLM output as a post process. You can also allow Guardrails to make re-asks to the LLM by specifying the num_reasks argument, or keep it purely as a post-processor by setting it to zero. Guard.parse returns the same fields as `__call__`.



In [20]:
output = groq_mistral_model.invoke("How to make a cake? please respond concisely in markdown format").content

print (output)

# How to Make a Cake

## Ingredients

- 2 cups all-purpose flour
- 1 cup granulated sugar
- 1/2 cup unsalted butter, softened
- 1 cup milk
- 3.5 teaspoons baking powder
- 1 teaspoon salt
- 2 teaspoons vanilla extract
- 2 eggs

## Instructions

1. Preheat the oven to 350°F (175°C). Grease and flour a 9x9 inch cake pan.
2. In a large bowl, beat the sugar and butter together until it's light and fluffy.
3. Beat in the eggs, one at a time, ensuring each one is incorporated before adding the next.
4. Stir in the vanilla extract.
5. In a separate bowl, combine the flour, baking powder, and salt.
6. Gradually add the dry ingredients to the butter mixture, alternating with the milk. Begin and end with the dry ingredients. Mix just until the batter is smooth.
7. Pour the batter into the prepared cake pan.
8. Bake for 30-35 minutes or until a toothpick inserted into the center comes out clean.
9. Allow the cake to cool in the pan for 10 minutes, then remove it from the pan and transfer it to a w

In [22]:
res = guard.parse(
    llm_output=output
)

print(res.validated_output) # Validated output



# How to Make a Cake

## Ingredients

- 2 cups all-purpose flour
- 1 cup granulated sugar
- 1/2 cup unsalted butter, softened
- 1 cup milk
- 3.5 teaspoons baking powder
- 1 teaspoon salt
- 2 teaspoons vanilla extract
- 2 eggs

## Instructions

1. Preheat the oven to 350°F (175°C). Grease and flour a 9x9 inch cake pan.
2. In a large bowl, beat the sugar and butter together until it's light and fluffy.
3. Beat in the eggs, one at a time, ensuring each one is incorporated before adding the next.
4. Stir in the vanilla extract.
5. In a separate bowl, combine the flour, baking powder, and salt.
6. Gradually add the dry ingredients to the butter mixture, alternating with the milk. Begin and end with the dry ingredients. Mix just until the batter is smooth.
7. Pour the batter into the prepared cake pan.
8. Bake for 30-35 minutes or until a toothpick inserted into the center comes out clean.
9. Allow the cake to cool in the pan for 10 minutes, then remove it from the pan and transfer it to a w

In [23]:
res = guard.parse(
    llm_output=output,
    model="mixtral-8x7b-32768",
    num_reasks=1
)

In [24]:
print(res.validated_output) # Validated output
print(guard.history.last.reasks) # A list of reasks

# How to Make a Cake

## Ingredients

- 2 cups all-purpose flour
- 1 cup granulated sugar
- 1/2 cup unsalted butter, softened
- 1 cup milk
- 3.5 teaspoons baking powder
- 1 teaspoon salt
- 2 teaspoons vanilla extract
- 2 eggs

## Instructions

1. Preheat the oven to 350°F (175°C). Grease and flour a 9x9 inch cake pan.
2. In a large bowl, beat the sugar and butter together until it's light and fluffy.
3. Beat in the eggs, one at a time, ensuring each one is incorporated before adding the next.
4. Stir in the vanilla extract.
5. In a separate bowl, combine the flour, baking powder, and salt.
6. Gradually add the dry ingredients to the butter mixture, alternating with the milk. Begin and end with the dry ingredients. Mix just until the batter is smooth.
7. Pour the batter into the prepared cake pan.
8. Bake for 30-35 minutes or until a toothpick inserted into the center comes out clean.
9. Allow the cake to cool in the pan for 10 minutes, then remove it from the pan and transfer it to a w